# Module 4 Graded Quiz

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:

def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time period t
    and r is the per-period interest rate
    returns a |t| x |r| Series or DataFrame
    r can be a float, Series or DataFrame
    returns a DataFrame indexed by t
    """
    discounts = pd.DataFrame([(r+1)**-i for i in t])
    discounts.index = t
    return discounts


def pv(flows, r):
    """
    Compute the present value of a sequence of cash flows given by the time (as an index) and amounts
    r can be a scalar, or a Series or DataFrame with the number of rows matching the num of rows in flows
    """
    dates = flows.index
    discounts = discount(dates, r)
    return discounts.multiply(flows, axis='rows').sum()

def bond_cash_flows(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12):
    """
    Returns the series of cash flows generated by a bond,
    indexed by the payment/coupon number
    """
    n_coupons = round(maturity*coupons_per_year)
    coupon_amt = principal*coupon_rate/coupons_per_year
    coupon_times = np.arange(1, n_coupons+1)
    cash_flows = pd.Series(data=coupon_amt, index=coupon_times)
    cash_flows.iloc[-1] += principal # add the principal to the last payment
    return cash_flows


def bond_price(maturity, principal=100, coupon_rate=0.03, coupons_per_year=12, discount_rate=0.03):
    """
    Computes the price of a bond that pays regular coupons until maturity
    at which time the principal and the final coupon is returned
    This is not designed to be efficient, rather,
    it is to illustrate the underlying principle behind bond pricing!
    If discount_rate is a DataFrame, then this is assumed to be the rate on each coupon date
    and the bond value is computed over time.
    i.e. The index of the discount_rate DataFrame is assumed to be the coupon number
    """
    if isinstance(discount_rate, pd.DataFrame):
        pricing_dates = discount_rate.index
        prices = pd.DataFrame(index=pricing_dates, columns=discount_rate.columns)
        for t in pricing_dates:
            prices.loc[t] = bond_price(maturity-t/coupons_per_year, principal, coupon_rate, coupons_per_year,
                                      discount_rate.loc[t])
        return prices
    else: # base case ... single time period
        if maturity <= 0: return principal+principal*coupon_rate/coupons_per_year
        cash_flows = bond_cash_flows(maturity, principal, coupon_rate, coupons_per_year)
        return pv(cash_flows, discount_rate/coupons_per_year)

In [3]:
b1_price=bond_price(15,1000,0.05,2,0.05)
b2_price=bond_price(5,1000,0.06,4,0.05)
b3_price=bond_price(10,1000,0,1,0.05)
print(f'B1 price:{b1_price.iloc[0]:.2f}, B2 price: {b2_price.iloc[0]:.2f}, B3 price:{b3_price.iloc[0]:.2f}')

B1 price:1000.00, B2 price: 1044.00, B3 price:613.91


In [4]:
#Q1:B2

In [5]:
#Q2:B3

In [6]:
#Q3:614

In [7]:

def macaulay_duration(flows, discount_rate):
    """
    Computes the Macaulay Duration of a sequence of cash flows, given a per-period discount rate
    """
    discounted_flows = discount(flows.index, discount_rate)*pd.DataFrame(flows)
    weights = discounted_flows/discounted_flows.sum()
    return np.average(flows.index, weights=weights.iloc[:,0])


In [8]:
B1_flows=bond_cash_flows(15,1000,0.05,2)
B2_flows=bond_cash_flows(5,1000,0.06,4)
B3_flows=bond_cash_flows(10,1000,0,1)

In [9]:
B1_duration=macaulay_duration(B1_flows,0.05/2)/2
B2_duration=macaulay_duration(B2_flows,0.05/4)/4
B3_duration=macaulay_duration(B3_flows,0.05)

In [10]:
print(f'B1 duration: {B1_duration:.2f}, B2 duration: {B2_duration:.2f}, B3 duration: {B3_duration:.2f}')

B1 duration: 10.73, B2 duration: 4.37, B3 duration: 10.00


In [11]:
#Q4:B1 

In [12]:
#Q5:B2

In [13]:
#Q6:4.37

In [14]:
liabilities=pd.Series(data=[100000,200000,300000], index=[3,5,10])

In [15]:
L_duration=macaulay_duration(liabilities,0.05)
print(f' Liabilities duration: {L_duration:.2f}')

 Liabilities duration: 6.75


In [16]:
#Q7:6.75

$$ w_s = \frac{d_l -d_t}{d_l - d_s} $$

In [17]:
print(f'Ws: {(B1_duration-L_duration)/(B1_duration-B2_duration)*100:.2f}')

Ws: 62.58


In [18]:
#Q8:62.58

In [19]:
#Q9: B1+B3 (because both durations are bigger than the liabilities duration)

In [20]:
print(f'Ws: {(B3_duration-L_duration)/(B3_duration-B2_duration)*100:.2f}')

Ws: 57.74


In [21]:
#Q10:57.74